# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# create file path
infile = os.path.join("Output", "city_weather_data.csv")

#read csv into a dataframe
city_data = pd.read_csv(infile)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

# Create a variable for lat, long
city_locations = city_data[["Lat", "Lng"]]

# Create a variable for humidity'
city_humidity = city_data["Humidity"]

# Create the  figure
figure_layout = {
    'width': '800px',
    'height': '580px',
    'border': '3px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(zoom_level = 2, center = (24.288292, 11.701573), layout=figure_layout)

# Create the heatmap layer for city locations and humidity
heat_layer = gmaps.heatmap_layer(city_locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=city_humidity.max(),
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(border='3px solid black', height='580px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
city_data_filtered = city_data.loc[(city_data['Max Temp'].between(70, 80)) & (city_data['Wind Speed'] < 5) & (city_data['Cloudiness'] < 5)].copy()  
city_data_clean = city_data_filtered.dropna()
city_data_clean

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
137,Mahenge,-8.68,36.72,73.83,51,1,3.40,TZ,1600359393
236,Tongzhou,39.91,116.60,75.00,76,0,4.47,CN,1600359446
262,Yerköy,39.64,34.47,76.23,20,2,3.87,TR,1600359454
296,Biak,-0.91,122.88,76.01,73,2,1.90,ID,1600359475
499,Ritchie,-29.04,24.60,77.00,9,0,1.12,ZA,1600359593


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = city_data_clean
hotel_df = hotel_df.reset_index(drop=True)

In [6]:
hotel_df['Hotel Name'] = ""
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Create a dictionary variable to store the parameters
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key,
}

for index, row in hotel_df.iterrows():

    # get restaurant type from DataFrame
    lat_longs = f"{row['Lat']},{row['Lng']}"

    # add a new key to our params dictionary that will store our keyword to search for
    params['location'] = lat_longs

    # assemble url and make API request
#    print(f"Retrieving Results for Index {index}: {row['City']} {lat_longs}.")
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"The closest hotel in {row['City']} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print(f"No hotel found in {row['City']}")

hotel_df

The closest hotel in Mahenge is Mkondosii Guest House.
The closest hotel in Tongzhou is Inner Mongolia Hotel.
The closest hotel in Yerköy is Yerköy Öğretmenevi.
The closest hotel in Biak is Kost Ariz.
The closest hotel in Ritchie is Riverside country club.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mahenge,-8.68,36.72,73.83,51,1,3.40,TZ,1600359393,Mkondosii Guest House
1,Tongzhou,39.91,116.60,75.00,76,0,4.47,CN,1600359446,Inner Mongolia Hotel
2,Yerköy,39.64,34.47,76.23,20,2,3.87,TR,1600359454,Yerköy Öğretmenevi
3,Biak,-0.91,122.88,76.01,73,2,1.90,ID,1600359475,Kost Ariz
4,Ritchie,-29.04,24.60,77.00,9,0,1.12,ZA,1600359593,Riverside country club


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the figure
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='3px solid black', height='580px', margin='0 auto 0 auto', padding='1px', wi…